In [1]:
# Import necessary libraries for data manipulation, plotting, and network analysis
import networkx as nx  # For handling graph data structures
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation using DataFrames
import logging  # For logging messages
import random  # For generating random numbers
import warnings
import os
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import json

#import GridCalEngine.api as gce  # For interfacing with the GridCal API
#import GridCalEngine.Devices as dev
#import GridCalEngine.Simulations as sim
#from GridCalEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
#from GridCalEngine.IO.file_handler import FileOpen
#import GridCalEngine.enumerations as en

import VeraGridEngine.api as gce  # For interfacing with the GridCal API
import VeraGridEngine.Devices as dev
import VeraGridEngine.Simulations as sim
from VeraGridEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
from VeraGridEngine.IO.file_handler import FileOpen
import VeraGridEngine.enumerations as en

import pandapower as pp
import simbench as sb
import pandapower.topology as top  # For topology analysis in Pandapower
import pandapower.plotting as plot  # For plotting in Pandapower
import pandapower.networks as nw

import src.GC_PandaPowerImporter as GC_PandaPowerImporter

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

In [2]:
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP = sb.get_simbench_net(sb_code1)
gridPP.switch.drop([232,234,236,238,240, 242,244,246], inplace=True)
gridPP.ext_grid.at[0,'name']="grid_ext"
gridPP.line['in_service'] = True

#print(gridPP.ext_grid)
#print(gridPP.bus.loc[gridPP.ext_grid.bus])

#pp.topology.create_nxgraph(gridPP)
#print(pp.topology.connected_components(gridPP))

#print(gridPP.switch.head())
#print(len(gridPP.switch))

#pp.diagnostic(net=gridPP)

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [3]:
import pandapower as pp
import networkx as nx

# 1) make sure gridPP is loaded
G = pp.topology.create_nxgraph(gridPP)
cc = list(nx.connected_components(G))
print("Number of connected components:", len(cc))
for i, comp in enumerate(cc):
    print(f"component {i}: total buses = {len(comp)}; example buses = {sorted(list(comp))[:20]}")
# ext_grid location
ext_bus = int(gridPP.ext_grid.at[0,'bus'])
for i, comp in enumerate(cc):
    if ext_bus in comp:
        print("ext_grid (bus {}) is in component {}".format(ext_bus, i))
        break
else:
    print("ext_grid's bus not found in any component!")

# 2) check r/x/length is/not 0 or super small
lines = gridPP.line.copy()
rcol = 'r_ohm_per_km' if 'r_ohm_per_km' in lines.columns else 'r_ohm_per_km'
xcol = 'x_ohm_per_km' if 'x_ohm_per_km' in lines.columns else 'x_ohm_per_km'
lencol = 'length_km' if 'length_km' in lines.columns else 'length_km'

zero_imp = lines[(lines[rcol].fillna(0).abs() < 1e-12) & (lines[xcol].fillna(0).abs() < 1e-12)]
zero_len = lines[lines[lencol].fillna(1e9) == 0]
very_small = lines[(lines[rcol].fillna(0).abs() < 1e-6) & (lines[xcol].fillna(0).abs() < 1e-6)]

print("Lines with r==0 & x==0:", zero_imp.index.tolist())
print("Lines with length == 0:", zero_len.index.tolist())
print("Lines with very small r/x:", very_small.index.tolist()[:80])

# 3) check trafo para
print(gridPP.trafo[['vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent']])
print("Transformers with vk_percent > 15:", gridPP.trafo[gridPP.trafo['vk_percent']>15].index.tolist())

# 4) check existance of lonely buses
connected_buses = set()
if 'from_bus' in gridPP.line.columns and 'to_bus' in gridPP.line.columns:
    connected_buses |= set(gridPP.line['from_bus'].unique())
    connected_buses |= set(gridPP.line['to_bus'].unique())
if 'hv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['hv_bus'].unique())
if 'lv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['lv_bus'].unique())

isolated = sorted(list(set(gridPP.bus.index) - connected_buses))
print("Number of isolated buses (no line/trafo connection):", len(isolated))
print("Some isolated buses:", isolated[:80])

# 5) list in_service=False line/trafo
print("Lines out-of-service count:", int((gridPP.line['in_service']==False).sum()))
print("Example lines out-of-service:", gridPP.line[gridPP.line['in_service']==False].index.tolist()[:40])
print("Trafos out-of-service count:", int((gridPP.trafo['in_service']==False).sum()))
print("Example trafos out-of-service:", gridPP.trafo[gridPP.trafo['in_service']==False].index.tolist()[:40])


Number of connected components: 1
component 0: total buses = 196; example buses = [np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]
ext_grid (bus 361) is in component 0
Lines with r==0 & x==0: []
Lines with length == 0: []
Lines with very small r/x: []
    vn_hv_kv   vn_lv_kv  vk_percent  vkr_percent
0 110.000000  20.000000   16.200000     0.340000
1 110.000000  20.000000   16.200000     0.340000
2 220.000000 110.000000   12.000000     0.128333
3 220.000000 110.000000   12.000000     0.128333
4 220.000000 110.000000   12.000000     0.128333
Transformers with vk_percent > 15: [0, 1]
Number of isolated buses (no line/trafo connection): 0
Some isolated buses: []
Lines out-of-service count: 0
Example lines out-of-service: []
Trafos out-of-service count: 0
Example trafos o

In [4]:
# 1) check injection
print("loads: count", len(gridPP.load), " total P MW:", gridPP.load['p_mw'].sum(), " min/max P_mw:", gridPP.load['p_mw'].min(), gridPP.load['p_mw'].max())
print(gridPP.load[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("sgen: count", len(gridPP.sgen), " total P MW:", gridPP.sgen['p_mw'].sum(), " min/max P_mw:", gridPP.sgen['p_mw'].min(), gridPP.sgen['p_mw'].max())
print(gridPP.sgen[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("gen: count", len(gridPP.gen), " total P MW:", gridPP.gen['p_mw'].sum() if len(gridPP.gen)>0 else 0)
print("ext_grid entries:", gridPP.ext_grid)

loads: count 194  total P MW: 521.7199999999999  min/max P_mw: 0.08 49.71
     bus      p_mw    q_mvar
193  560 49.710000 19.647000
192  448 49.710000 19.647000
181  526 34.480000 13.627000
182  528 34.480000 13.627000
183  643 34.480000 13.627000
189  451 31.640000 12.505000
190  675 31.640000 12.505000
191  655 31.640000 12.505000
186  569 17.260000  6.822000
184  424 17.260000  6.822000
sgen: count 219  total P MW: 609.9989999999999  min/max P_mw: 0.0 46.22
     bus      p_mw   q_mvar
215  451 46.220000 0.000000
208  414 33.480000 0.000000
217  619 33.480000 0.000000
190  424 25.570000 0.000000
192  569 25.570000 0.000000
193  669 25.570000 0.000000
191  461 25.570000 0.000000
194  712 25.570000 0.000000
212  586 25.070000 0.000000
196  675 23.800000 0.000000
gen: count 0  total P MW: 0
ext_grid entries:        name  bus    vm_pu  va_degree  slack_weight  in_service controllable voltLvl sn_mva min_q_mvar p_disp_mw profile max_p_mw max_q_mvar type    phys_type min_p_mw       subnet
0

In [5]:
# 2) test
import pandapower as pp

# try A: NR + DC init + larger iteration number
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged")
except Exception as e:
    print("nr+dc failed:", repr(e))

# try B: BFSW
try:
    pp.runpp(gridPP, algorithm='bfsw')
    print("bfsw converged")
except Exception as e:
    print("bfsw failed:", repr(e))

nr+dc failed: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
bfsw failed: ValueError('negative axis 1 index: -1')


In [6]:
# 3) temp change trafo: vk_percent
# backup rn value
orig_vk = gridPP.trafo['vk_percent'].copy()
# turn all vk_percent > 15 trafo to 15 (temp)
gridPP.trafo.loc[gridPP.trafo['vk_percent']>15, 'vk_percent'] = 15.0
print("changed vk_percent for trafos:", gridPP.trafo.loc[gridPP.trafo['vk_percent']==15].index.tolist())

# try again nr+dc
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged after lowering vk_percent")
except Exception as e:
    print("nr+dc still failed after lowering vk_percent:", repr(e))

# back to orginal value
gridPP.trafo['vk_percent'] = orig_vk

changed vk_percent for trafos: [0, 1]
nr+dc still failed after lowering vk_percent: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [7]:
# 0 injection
# backup
loads_p = gridPP.load['p_mw'].copy()
loads_q = gridPP.load['q_mvar'].copy()
sgen_p  = gridPP.sgen['p_mw'].copy()
sgen_q  = gridPP.sgen['q_mvar'].copy()

# 0 all load and sgen
gridPP.load.loc[:, ['p_mw','q_mvar']] = 0.0
gridPP.sgen.loc[:, ['p_mw','q_mvar']] = 0.0

# run NR + dc init, bigger iter
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("ZERO-INJECTION runpp: CONVERGED")
except Exception as e:
    print("ZERO-INJECTION runpp: FAILED ->", repr(e))

# recover
gridPP.load['p_mw'] = loads_p
gridPP.load['q_mvar'] = loads_q
gridPP.sgen['p_mw'] = sgen_p
gridPP.sgen['q_mvar'] = sgen_q

ZERO-INJECTION runpp: FAILED -> LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [8]:
# Failed
print("lines r_ohm_per_km min/max:", gridPP.line['r_ohm_per_km'].min(), gridPP.line['r_ohm_per_km'].max())
print("lines x_ohm_per_km min/max:", gridPP.line['x_ohm_per_km'].min(), gridPP.line['x_ohm_per_km'].max())
print("lines length_km min/max:", gridPP.line['length_km'].min(), gridPP.line['length_km'].max())
# show extreme
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('r_ohm_per_km').head(10))
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('x_ohm_per_km').head(10))
print(gridPP.trafo[['hv_bus','lv_bus','vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent','parallel']])
pairs = gridPP.line.groupby(['from_bus','to_bus']).size()
dup_pairs = pairs[pairs>1]
print("Duplicate (from,to) line pairs:", dup_pairs.head(20))

lines r_ohm_per_km min/max: 0.1095 0.5939
lines x_ohm_per_km min/max: 0.119 0.372
lines length_km min/max: 0.15 51.636
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
121       388     537      0.109500      0.296000   4.682650
122       473     704      0.109500      0.296000   5.839610
123       473     704      0.109500      0.296000   5.839610
127       424     430      0.109500      0.296000   3.192820
220       398     586      0.109500      0.296000   7.133880
219       443     619      0.109500      0.296000   7.332100
217       560     455      0.109500      0.296000   7.122820
222       408     473      0.109500      0.296000   8.854110
216       661     678      0.109500      0.296000  11.166600
211       398     458      0.109500      0.296000  30.555000
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
23         24      25      0.253000      0.119000   0.500000
24         25      26      0.253000      0.119000   0.350000
22          0      24      

In [9]:
# PLAN A
# One-time temporary disabling of extra lines and redundant parallel transformers in a "duplicate pair" (quick verification)
import pandapower as pp
from collections import defaultdict

# backup
line_in_service_backup = gridPP.line['in_service'].copy()
trafo_in_service_backup = gridPP.trafo['in_service'].copy()

# Find duplicate (from_bus, to_bus) pairs (consider only duplicates)
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

# Center each duplicate pair, keep the first one, and set the others to in_service=False
disabled_lines = []
for k, idx_list in dup_pairs.items():
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
        disabled_lines.append(idx)

# For trafo: find duplicate hv_bus/lv_bus (parallel trafo)
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

disabled_trafos = []
for k, idx_list in dup_trafos.items():
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
        disabled_trafos.append(idx)

print("Temporarily disabled line indices (duplicates):", disabled_lines[:80])
print("Temporarily disabled trafo indices (duplicates):", disabled_trafos)

# run pf
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("runpp succeeded AFTER disabling duplicate extra branches.")
    conv_success = True
except Exception as e:
    print("runpp STILL failed after disabling duplicates:", repr(e))
    conv_success = False

# Restore the original in_service state (whether successful or not)
gridPP.line['in_service'] = line_in_service_backup
gridPP.trafo['in_service'] = trafo_in_service_backup

print("Restored original in_service flags.")

Temporarily disabled line indices (duplicates): [233, 208, 227, 232, 140, 225, 231, 151, 186, 173, 123, 226, 187, 178, 170, 139, 166, 161, 228, 188, 229, 180, 223, 230]
Temporarily disabled trafo indices (duplicates): [3, 4, 1]
runpp STILL failed after disabling duplicates: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
Restored original in_service flags.


In [10]:
# PLAN B
# If disabling all extra branches for all repetitions at once still doesn't converge, test each pair of repetitions to determine which pair is causing the problem. The following script disables the extra entries for each pair of repetitions (lines or trafo) and tests them, outputting the test results for each pair, thus pinpointing the problematic pairs.
import pandapower as pp

# backup
line_backup = gridPP.line['in_service'].copy()
trafo_backup = gridPP.trafo['in_service'].copy()

problematic_line_pairs = []
# Test each repeated line pair/group separately
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

for k, idx_list in dup_pairs.items():
    # disable all extras for this pair only, leave others as original
    gridPP.line['in_service'] = line_backup.copy()
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing pair {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_line_pairs.append((k, idx_list))

# restore
gridPP.line['in_service'] = line_backup

# Then do the same test on the trafo parallel group
problematic_trafo_pairs = []
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

for k, idx_list in dup_trafos.items():
    gridPP.trafo['in_service'] = trafo_backup.copy()
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing trafo group {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_trafo_pairs.append((k, idx_list))

# restore
gridPP.trafo['in_service'] = trafo_backup

print("Problematic line pairs (that STILL fail even when extras disabled):", problematic_line_pairs)
print("Problematic trafo groups:", problematic_trafo_pairs)

Testing pair (372, 418), indices [133, 233] -> converged? False
Testing pair (372, 655), indices [124, 208] -> converged? False
Testing pair (372, 704), indices [143, 227, 232] -> converged? False
Testing pair (392, 571), indices [136, 140] -> converged? False
Testing pair (392, 594), indices [134, 225] -> converged? False
Testing pair (418, 424), indices [181, 231] -> converged? False
Testing pair (424, 430), indices [127, 151] -> converged? False
Testing pair (430, 520), indices [125, 186] -> converged? False
Testing pair (473, 601), indices [129, 173] -> converged? False
Testing pair (473, 704), indices [122, 123, 226] -> converged? False
Testing pair (506, 526), indices [132, 187] -> converged? False
Testing pair (506, 645), indices [177, 178] -> converged? False
Testing pair (520, 526), indices [128, 170] -> converged? False
Testing pair (540, 594), indices [131, 139] -> converged? False
Testing pair (540, 601), indices [130, 166] -> converged? False
Testing pair (552, 630), indic

In [11]:
# One-click cleaning (merging duplicate lines & parallel transformers + minor disturbances + verification)
# paste this into a cell in your notebook (run on the same kernel where gridPP is loaded)
import pandas as pd
import numpy as np
import pandapower as pp

def repair_pandapower_net(net, verbose=True):
    repaired = False

    # -------- Duplicate lines: group by from/to/std_type/length_km -----------
    if 'std_type' not in net.line.columns:
        net.line['std_type'] = net.line.get('std_type', pd.Series([None]*len(net.line)))
    line_group_cols = ["from_bus", "to_bus", "std_type", "length_km"]
    grouped = net.line.groupby(line_group_cols).size().reset_index(name="count")
    dup_lines = grouped[grouped["count"] > 1]

    if not dup_lines.empty:
        repaired = True
        if verbose: print(f"Found {len(dup_lines)} duplicate line groups -> merging them...")
        new_lines = []
        for _, r in dup_lines.iterrows():
            mask = (
                (net.line.from_bus == r.from_bus) &
                (net.line.to_bus == r.to_bus) &
                (net.line.std_type == r.std_type) &
                (net.line.length_km == r.length_km)
            )
            sub = net.line[mask]
            # Keep first entry as template; sum nothing (we keep line geometry),
            # but you can also combine max_i_ka or number of parallel conductors if meaningful.
            base = sub.iloc[0].copy()
            base['max_i_ka'] = sub['max_i_ka'].max()
            base['in_service']  = sub['in_service'].any()
            new_lines.append(base)
            # drop all original duplicates
            net.line.drop(sub.index, inplace=True)
        # append merged ones
        net.line = pd.concat([net.line, pd.DataFrame(new_lines)], ignore_index=True)
        if verbose: print("Merged duplicate lines. New line count:", len(net.line))

    # -------- Duplicate trafos (parallel) -----------
    trafo_group_cols = ["hv_bus", "lv_bus", "vn_hv_kv", "vn_lv_kv"]
    if set(trafo_group_cols).issubset(net.trafo.columns):
        grouped_t = net.trafo.groupby(trafo_group_cols).size().reset_index(name="count")
        dup_trafos = grouped_t[grouped_t["count"] > 1]
        if not dup_trafos.empty:
            repaired = True
            if verbose: print(f"Found {len(dup_trafos)} duplicate trafo groups -> merging them...")
            new_trafos = []
            for _, r in dup_trafos.iterrows():
                mask = (
                    (net.trafo.hv_bus == r.hv_bus) &
                    (net.trafo.lv_bus == r.lv_bus) &
                    (net.trafo.vn_hv_kv == r.vn_hv_kv) &
                    (net.trafo.vn_lv_kv == r.vn_lv_kv)
                )
                sub = net.trafo[mask]
                base = sub.iloc[0].copy()
                # sum rated power
                if 'sn_mva' in sub.columns:
                    base['sn_mva'] = sub['sn_mva'].sum()
                base['in_service'] = sub['in_service'].any()
                new_trafos.append(base)
                net.trafo.drop(sub.index, inplace=True)
            net.trafo = pd.concat([net.trafo, pd.DataFrame(new_trafos)], ignore_index=True)
            if verbose: print("Merged duplicate trafos. New trafo count:", len(net.trafo))
    else:
        if verbose: print("Trafo columns missing expected fields; skipping trafo merge.")

    # -------- Tiny random perturbation to break exact symmetry -------------
    # (only if needed; harmless small relative changes)
    rng = np.random.default_rng(seed=12345)
    # ensure columns exist
    if 'r_ohm_per_km' in net.line.columns:
        net.line['r_ohm_per_km'] *= (1 + rng.uniform(-1e-6, 1e-6, size=len(net.line)))
    if 'x_ohm_per_km' in net.line.columns:
        net.line['x_ohm_per_km'] *= (1 + rng.uniform(-1e-6, 1e-6, size=len(net.line)))
    if 'vk_percent' in net.trafo.columns:
        net.trafo['vk_percent'] *= (1 + rng.uniform(-1e-6, 1e-6, size=len(net.trafo)))

    # -------- Try to run power flow ----------
    try:
        pp.runpp(net, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        if verbose: print("Power flow converged after repair.")
        converged = True
    except Exception as e:
        if verbose: print("Power flow STILL failed after repair:", type(e), e)
        converged = False

    return net, converged, repaired

# USAGE:
# gridPP is your simbench net (already loaded)
gridPP_copy = gridPP.deepcopy()   # keep a safe copy
gridPP_clean, ok, changed = repair_pandapower_net(gridPP_copy, verbose=True)
print("Converged?", ok, "Repaired anything?", changed)
# If ok True you can use gridPP_clean for the rest of your notebook.


Found 20 duplicate line groups -> merging them...
Merged duplicate lines. New line count: 212
Found 2 duplicate trafo groups -> merging them...
Merged duplicate trafos. New trafo count: 2
Power flow STILL failed after repair: <class 'pandapower.auxiliary.LoadflowNotConverged'> Power Flow nr did not converge after 200 iterations!
Converged? False Repaired anything? True


In [12]:
import pandapower as pp
import numpy as np

def test_minimal_convergent_subnet(net, verbose=True):
    net_test = net.deepcopy()
    results = {}

    # 1️⃣ 关闭所有 sgen，只留外部网
    net_test.sgen["in_service"] = False
    net_test.gen["in_service"] = False
    try:
        pp.runpp(net_test, algorithm="nr", init="dc", max_iteration=200, tolerance_mva=1e-8)
        results["only_ext_grid"] = True
        if verbose: print("✅ Converged with only external grid.")
    except Exception as e:
        results["only_ext_grid"] = False
        if verbose: print("❌ Still failed with only external grid:", e)

    # 2️⃣ 如果成功，再逐步恢复一部分 sgen
    if results["only_ext_grid"]:
        sgen_indices = net.sgen.index.tolist()
        chunk = max(1, len(sgen_indices)//5)
        for i in range(1,6):
            idxs = sgen_indices[:i*chunk]
            net_test.sgen.loc[idxs, "in_service"] = True
            try:
                pp.runpp(net_test, algorithm="nr", init="dc", max_iteration=200)
                results[f"sgen_{i*20}%"] = True
                if verbose: print(f"✅ Converged with {i*20}% of sgens active")
            except Exception:
                results[f"sgen_{i*20}%"] = False
                if verbose: print(f"❌ Failed with {i*20}% of sgens active")
                break

    # 3️⃣ 若只靠 ext_grid 也不行，再缩放所有负荷和发电
    if not results.get("only_ext_grid", False):
        for scale in [0.1, 0.01, 0.001]:
            net_scaled = net.deepcopy()
            net_scaled.load["p_mw"] *= scale
            net_scaled.load["q_mvar"] *= scale
            net_scaled.sgen["p_mw"] *= scale
            try:
                pp.runpp(net_scaled, algorithm="nr", init="dc", max_iteration=200)
                if verbose: print(f"✅ Converged when scaling P,Q by {scale}")
                results["scale_factor"] = scale
                break
            except Exception:
                if verbose: print(f"❌ Failed when scaling P,Q by {scale}")
                continue

    return results

results = test_minimal_convergent_subnet(gridPP_clean, verbose=True)
print("\nSummary of results:", results)


❌ Still failed with only external grid: Power Flow nr did not converge after 200 iterations!
❌ Failed when scaling P,Q by 0.1
❌ Failed when scaling P,Q by 0.01
❌ Failed when scaling P,Q by 0.001

Summary of results: {'only_ext_grid': False}


In [13]:
print(gridPP_clean.trafo[["hv_bus","lv_bus","sn_mva","vk_percent","vkr_percent","in_service"]])


   hv_bus  lv_bus     sn_mva  vk_percent  vkr_percent  in_service
0     361     372 900.000000   11.999993     0.128333        True
1     655       0  80.000000   16.200007     0.340000        True


In [14]:
gridPP_clean.trafo.loc[gridPP_clean.trafo.vn_lv_kv == 110, "vk_percent"] *= 1.5
pp.runpp(gridPP_clean, algorithm="nr", init="dc", max_iteration=200)


LoadflowNotConverged: Power Flow nr did not converge after 200 iterations!

In [15]:
import pandapower as pp

net_fix = gridPP_clean.deepcopy()
# 增加一点变压器阻抗
net_fix.trafo.loc[:, "vk_percent"] *= 5
net_fix.trafo.loc[:, "vkr_percent"] *= 5

try:
    pp.runpp(net_fix, algorithm="nr", init="dc", max_iteration=200, tolerance_mva=1e-8)
    print("✅ Power flow converged after increasing trafo impedance!")
except Exception as e:
    print("❌ Still failed after impedance increase:", e)


❌ Still failed after impedance increase: Power Flow nr did not converge after 200 iterations!


In [16]:
net_test = gridPP_clean.deepcopy()
net_test.trafo.loc[0, "in_service"] = False  # 暂时断开 (361,372)
try:
    pp.runpp(net_test, algorithm="nr", init="dc")
    print("✅ Converged after disconnecting trafo (361,372)")
except Exception as e:
    print("❌ Still failed:", e)


✅ Converged after disconnecting trafo (361,372)


In [18]:
import pandapower.topology as top
import networkx as nx
import pandapower as pp

# 1️⃣ 创建 networkx 图
G = top.create_nxgraph(gridPP_clean, include_trafos=True)

# 2️⃣ 提取 372 所在子网
ccs = list(nx.connected_components(G))
for comp in ccs:
    if 372 in comp:
        sub = G.subgraph(comp).copy()
        break

# 3️⃣ 寻找环路
cycles = list(nx.simple_cycles(sub.to_directed()))
print(f"Found {len(cycles)} cycles near bus 372")

# 4️⃣ 输出最短环和对应线路
if cycles:
    shortest = min(cycles, key=len)
    print("Shortest cycle:", shortest)

    line_ids = []
    for fb, tb in zip(shortest, shortest[1:] + [shortest[0]]):
        mask = ((gridPP_clean.line.from_bus == fb) & (gridPP_clean.line.to_bus == tb)) | \
               ((gridPP_clean.line.from_bus == tb) & (gridPP_clean.line.to_bus == fb))
        ids = gridPP_clean.line.index[mask].tolist()
        line_ids.extend(ids)

    print("Lines in shortest cycle:", line_ids)
else:
    print("⚠️ No cycles detected near bus 372")


Found 358 cycles near bus 372
Shortest cycle: [np.int64(0), np.int64(2)]
Lines in shortest cycle: [0, 0]


In [19]:
import numpy as np

# 1️⃣ 找出 bus 0 和 bus 2 之间的所有线路
mask = ((gridPP_clean.line.from_bus == 0) & (gridPP_clean.line.to_bus == 2)) | \
       ((gridPP_clean.line.from_bus == 2) & (gridPP_clean.line.to_bus == 0))
dup_lines = gridPP_clean.line[mask]
print("Duplicate/parallel lines between 0 and 2:\n", dup_lines)

# 2️⃣ 删除其中一条（保留第一条）
if len(dup_lines) > 1:
    to_drop = dup_lines.index[1:]  # 除第一条以外的全部删除
    gridPP_clean.line.drop(index=to_drop, inplace=True)
    print(f"Dropped duplicate lines: {to_drop.tolist()}")

# 3️⃣ 检查该线阻抗是否过低
line = gridPP_clean.line.loc[dup_lines.index[0]]
print("Remaining line params:", line[['r_ohm_per_km', 'x_ohm_per_km', 'length_km']])

if line['r_ohm_per_km'] < 0.05 or line['x_ohm_per_km'] < 0.05:
    print("⚠️ Line impedance too low, adjusting to safe minimum...")
    gridPP_clean.line.at[dup_lines.index[0], 'r_ohm_per_km'] = max(0.05, line['r_ohm_per_km'])
    gridPP_clean.line.at[dup_lines.index[0], 'x_ohm_per_km'] = max(0.05, line['x_ohm_per_km'])
    gridPP_clean.line.at[dup_lines.index[0], 'length_km'] = max(0.1, line['length_km'])

# 4️⃣ 重新运行潮流
try:
    pp.runpp(gridPP_clean, init='flat', tolerance_mva=1e-6, max_iteration=200)
    print("✅ Power flow converged after fix.")
except Exception as e:
    print("❌ Still not converged:", e)


Duplicate/parallel lines between 0 and 2:
              name                     std_type  from_bus  to_bus  length_km  r_ohm_per_km  x_ohm_per_km  c_nf_per_km  g_us_per_km  max_i_ka       df  parallel type  in_service  geo voltLvl  max_loading_percent           subnet
0  MV2.203 Line 1  NA2XS2Y 1x95 RM/25 12/20 kV         0       2   0.700000      0.313000      0.132000   215.999996     0.000000  0.252000 1.000000         1   cs        True  NaN       5           100.000000  MV2.203_Feeder1
Remaining line params: r_ohm_per_km   0.313000
x_ohm_per_km   0.132000
length_km      0.700000
Name: 0, dtype: object
❌ Still not converged: Power Flow nr did not converge after 200 iterations!


In [24]:
import numpy as np
import pandapower.topology as top

net = gridPP_clean

print("🔍 Cleaning duplicate or near-zero impedance connections ...")

# Step 1: 去掉重复 (from_bus,to_bus)
dup_pairs = net.line.groupby(["from_bus", "to_bus"]).size()
dup_pairs = dup_pairs[dup_pairs > 1]
print(f"Found {len(dup_pairs)} duplicated line pairs")

for (f, t), count in dup_pairs.items():
    idxs = net.line.index[(net.line.from_bus == f) & (net.line.to_bus == t)]
    net.line.drop(index=idxs[1:], inplace=True)

# Step 2: 检查 r,x 是否过小
mask_low_imp = (net.line.r_ohm_per_km * net.line.length_km < 0.05) | \
               (net.line.x_ohm_per_km * net.line.length_km < 0.05)
low_lines = net.line[mask_low_imp]
if len(low_lines):
    print(f"⚠️ {len(low_lines)} lines with too small impedance, fixing them...")
    net.line.loc[low_lines.index, "r_ohm_per_km"] = 0.1
    net.line.loc[low_lines.index, "x_ohm_per_km"] = 0.1
    net.line.loc[low_lines.index, "length_km"] = 0.1

# Step 3: 清理 switch 里与断开线重复的项
if "switch" in net and len(net.switch):
    net.switch = net.switch.loc[net.switch.in_service | (net.switch.et != "l")]

# Step 4: 检查是否连通
G = top.create_nxgraph(gridPP_clean, include_trafos=True)

ccs = list(nx.connected_components(G))
print("Connected components:", len(ccs))
if len(ccs) > 1:
    print("⚠️ Network not fully connected — only using the component with ext_grid.")
    ext_bus = int(net.ext_grid.bus.iloc[0])
    comp_main = [c for c in ccs if ext_bus in c][0]
    buses_to_keep = set(comp_main)
    net.bus = net.bus.loc[net.bus.index.isin(buses_to_keep)]
    net.line = net.line.loc[net.line.from_bus.isin(buses_to_keep) & net.line.to_bus.isin(buses_to_keep)]

# Step 5: 再次潮流
try:
    pp.runpp(net, init='flat', tolerance_mva=1e-6, max_iteration=200)
    print("✅ Power flow converged after deep clean!")
except Exception as e:
    print("❌ Still not converged after deep clean:", e)


🔍 Cleaning duplicate or near-zero impedance connections ...
Found 0 duplicated line pairs
⚠️ 57 lines with too small impedance, fixing them...
Connected components: 1
❌ Still not converged after deep clean: Power Flow nr did not converge after 200 iterations!


In [25]:
import pandapower as pp
import pandapower.topology as top

net = gridPP_clean

# 1️⃣ 提取不同电压等级
voltage_levels = sorted(net.bus.vn_kv.unique())
print(f"Voltage levels in network: {voltage_levels}")

# 2️⃣ 遍历各电压等级，单独潮流
results = {}
for vn in voltage_levels:
    buses = net.bus.index[net.bus.vn_kv == vn]
    sub = pp.select_subnet(net, buses)
    print(f"\n⚙️ Testing subnet {vn} kV: {len(sub.bus)} buses, {len(sub.line)} lines")

    try:
        pp.runpp(sub, init='flat', max_iteration=100)
        results[vn] = True
        print(f"✅ Converged at {vn} kV")
    except Exception as e:
        results[vn] = False
        print(f"❌ Not converged at {vn} kV: {e}")

print("\nSummary by voltage level:")
for vn, ok in results.items():
    print(f"{vn:>8.2f} kV : {'✅' if ok else '❌'}")


Voltage levels in network: [np.float64(20.0), np.float64(110.0), np.float64(220.0)]

⚙️ Testing subnet 20.0 kV: 114 buses, 121 lines
❌ Not converged at 20.0 kV: No reference bus is available. Either add an ext_grid or a gen with slack=True

⚙️ Testing subnet 110.0 kV: 81 buses, 89 lines
❌ Not converged at 110.0 kV: No reference bus is available. Either add an ext_grid or a gen with slack=True

⚙️ Testing subnet 220.0 kV: 1 buses, 0 lines
✅ Converged at 220.0 kV

Summary by voltage level:
   20.00 kV : ❌
  110.00 kV : ❌
  220.00 kV : ✅


In [27]:
import pandapower as pp
import pandapower.topology as top
import networkx as nx

net = gridPP_clean

print("🔍 Checking transformer connectivity between voltage levels...\n")

for i, trafo in net.trafo.iterrows():
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    hv_v = net.bus.at[hv_bus, "vn_kv"]
    lv_v = net.bus.at[lv_bus, "vn_kv"]
    print(f"Trafo {i}: {hv_v:.1f} kV → {lv_v:.1f} kV, buses ({hv_bus}->{lv_bus})")

print("\n🧠 Building graph to test connectivity...")

# 创建网络图
G = top.create_nxgraph(net, include_trafos=True)
ext_bus = int(net.ext_grid.bus.iloc[0])

# 找出外部网可达的所有节点
reachable = nx.node_connected_component(G, ext_bus)

# 取20kV母线
mv_buses = net.bus.index[net.bus.vn_kv == 20.0].tolist()
unreachable_mv = [b for b in mv_buses if b not in reachable]

print(f"Total 20 kV buses: {len(mv_buses)}, unreachable: {len(unreachable_mv)}")

if len(unreachable_mv) > 0:
    print("⚠️ The 20 kV network seems isolated from the main grid!")
    test_bus = unreachable_mv[0]
    print(f"Adding temporary ext_grid at bus {test_bus} (20 kV) ...")
    pp.create_ext_grid(net, bus=test_bus, vm_pu=1.0, name="TEMP_MV_SOURCE")

try:
    pp.runpp(net, init='flat', max_iteration=200)
    print("✅ Power flow converged with or without temporary ext_grid.")
except Exception as e:
    print("❌ Still not converged even with temporary ext_grid:", e)



🔍 Checking transformer connectivity between voltage levels...

Trafo 0: 220.0 kV → 110.0 kV, buses (361->372)
Trafo 1: 110.0 kV → 20.0 kV, buses (655->0)

🧠 Building graph to test connectivity...
Total 20 kV buses: 114, unreachable: 0
❌ Still not converged even with temporary ext_grid: Power Flow nr did not converge after 200 iterations!


In [32]:
import pandapower as pp
import simbench as sb

sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP = sb.get_simbench_net(sb_code1, as_pp=True)  # 这是 pandapowerNet 对象
print(type(gridPP))
gridPP_clean = gridPP.copy()             # 做深拷贝用于清理
print(type(gridPP_clean))
# <class 'pandapower.pandapowerNet'>


TypeError: get_simbench_net() got an unexpected keyword argument 'as_pp'

In [34]:
import simbench as sb

sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
grid_dict = sb.get_simbench_net(sb_code1)

# 查看所有键
print("Available keys in the network dict:")
for k in grid_dict.keys():
    print(k)


Available keys in the network dict:
bus
bus_dc
load
sgen
motor
asymmetric_load
asymmetric_sgen
storage
gen
switch
shunt
svc
ssc
vsc
ext_grid
line
line_dc
trafo
trafo3w
impedance
tcsc
dcline
ward
xward
measurement
pwl_cost
poly_cost
controller
group
source_dc
load_dc
b2b_vsc
_empty_res_b2b_vsc
bi_vsc
_empty_res_bi_vsc
_empty_res_bus
_empty_res_bus_dc
_empty_res_ext_grid
_empty_res_line
_empty_res_line_dc
_empty_res_trafo
_empty_res_load
_empty_res_asymmetric_load
_empty_res_asymmetric_sgen
_empty_res_motor
_empty_res_sgen
_empty_res_shunt
_empty_res_svc
_empty_res_ssc
_empty_res_vsc
_empty_res_switch
_empty_res_impedance
_empty_res_tcsc
_empty_res_dcline
_empty_res_source_dc
_empty_res_load_dc
_empty_res_ward
_empty_res_xward
_empty_res_trafo_3ph
_empty_res_trafo3w
_empty_res_bus_3ph
_empty_res_ext_grid_3ph
_empty_res_line_3ph
_empty_res_asymmetric_load_3ph
_empty_res_asymmetric_sgen_3ph
_empty_res_storage
_empty_res_storage_3ph
_empty_res_gen
_empty_res_protection
_ppc
_ppc0
_ppc1
_ppc

In [38]:
import simbench as sb
import pandapower as pp
import pandapower.topology as top
import networkx as nx
import numpy as np

def D7_repair_and_powerflow(sb_code):
    print(f"🔧 Loading SimBench network: {sb_code}\n")

    # Step 0: 获取网络
    grid_dict = sb.get_simbench_net(sb_code)  # 你的版本返回 dict
    # 直接使用 simbench 的字典结构
    # 创建一个空 pandapowerNet，并填充数据
    net = pp.create_empty_network()

    # 填充 bus
    for _, row in grid_dict['bus'].iterrows():
        pp.create_bus(net, vn_kv=row.vn_kv, name=row.name, in_service=row.in_service)

    # 填充 line
    for _, row in grid_dict['line'].iterrows():
        pp.create_line_from_parameters(
            net,
            from_bus=int(row.from_bus),
            to_bus=int(row.to_bus),
            length_km=row.length_km if 'length_km' in row else 1.0,
            r_ohm_per_km=row.r_ohm_per_km if 'r_ohm_per_km' in row else 0.1,
            x_ohm_per_km=row.x_ohm_per_km if 'x_ohm_per_km' in row else 0.1,
            c_nf_per_km=row.c_nf_per_km if 'c_nf_per_km' in row else 0,
            max_i_ka=row.max_i_ka if 'max_i_ka' in row else 1.0,
            name=row.name if 'name' in row else None,
            in_service=row.in_service if 'in_service' in row else True
        )

    # 填充 trafo
    for _, row in grid_dict['trafo'].iterrows():
        pp.create_transformer_from_parameters(
            net,
            hv_bus=int(row.hv_bus),
            lv_bus=int(row.lv_bus),
            sn_mva=row.sn_mva,
            vn_hv_kv=row.vn_hv_kv,
            vn_lv_kv=row.vn_lv_kv,
            vk_percent=row.vk_percent,
            vkr_percent=row.vkr_percent,
            pfe_kw=row.pfe_kw if 'pfe_kw' in row else 0,
            i0_percent=row.i0_percent if 'i0_percent' in row else 0,
            name=row.name if 'name' in row else None,
            in_service=row.in_service if 'in_service' in row else True
        )

    # 填充 ext_grid（只取第一个）
    if len(grid_dict['ext_grid']):
        eg = grid_dict['ext_grid'].iloc[0]
        pp.create_ext_grid(net, bus=int(eg.bus), vm_pu=eg.vm_pu if 'vm_pu' in eg else 1.0, name=eg.name if 'name' in eg else "ext_grid")

    print(f"✅ Loaded network: buses={len(net.bus)}, lines={len(net.line)}, trafos={len(net.trafo)}")

    # ---------------------
    # Step 1: 去掉重复线路
    dup_pairs = net.line.groupby(["from_bus", "to_bus"]).size()
    dup_pairs = dup_pairs[dup_pairs > 1]
    print(f"Found {len(dup_pairs)} duplicated line pairs")
    for (f, t), count in dup_pairs.items():
        idxs = net.line.index[(net.line.from_bus == f) & (net.line.to_bus == t)]
        net.line.drop(index=idxs[1:], inplace=True)

    # Step 2: 修正低阻抗线路
    mask_low_imp = (net.line.r_ohm_per_km * net.line.length_km < 0.05) | \
                   (net.line.x_ohm_per_km * net.line.length_km < 0.05)
    low_lines = net.line[mask_low_imp]
    if len(low_lines):
        print(f"⚠️ Fixing {len(low_lines)} low-impedance lines...")
        net.line.loc[low_lines.index, "r_ohm_per_km"] = 0.1
        net.line.loc[low_lines.index, "x_ohm_per_km"] = 0.1
        net.line.loc[low_lines.index, "length_km"] = 0.1

    # Step 3: 清理开关
    if "switch" in grid_dict and len(grid_dict['switch']):
        net.switch = grid_dict['switch'][grid_dict['switch'].in_service | (grid_dict['switch'].et != "l")]

    # Step 4: 保留与外部网连通子网
    G = top.create_nxgraph(net, include_trafos=True)
    ccs = list(nx.connected_components(G))
    print(f"Connected components: {len(ccs)}")
    if len(ccs) > 1:
        ext_bus = int(net.ext_grid.bus.iloc[0])
        comp_main = [c for c in ccs if ext_bus in c][0]
        buses_to_keep = set(comp_main)
        net.bus = net.bus.loc[net.bus.index.isin(buses_to_keep)]
        net.line = net.line.loc[net.line.from_bus.isin(buses_to_keep) & net.line.to_bus.isin(buses_to_keep)]
        print(f"⚠️ Kept {len(buses_to_keep)} buses in main connected component")

    # Step 5: 潮流尝试
    try:
        pp.runpp(net, init='flat', tolerance_mva=1e-6, max_iteration=200)
        print("✅ Power flow converged!")
    except pp.LoadflowNotConverged as e:
        print("❌ Power flow did NOT converge:", e)

    return net

# --------------------------
# 使用示例
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP_clean = D7_repair_and_powerflow(sb_code1)


🔧 Loading SimBench network: 1-HVMV-urban-2.203-0-no_sw



TypeError: %d format: a real number is required, not NoneType

In [39]:
import simbench as sb
import pandapower as pp
import pandas as pd
import pandapower.topology as top
import networkx as nx

def D7_fix_v2(sb_code):
    print(f"🔧 Loading and repairing SimBench network: {sb_code}\n")

    # Step 0: 载入 SimBench 原始数据（返回的是字典）
    grid_dict = sb.get_simbench_net(sb_code)
    net = pp.create_empty_network()

    # Step 1: 添加 bus
    for i, row in grid_dict['bus'].iterrows():
        pp.create_bus(net, vn_kv=row.vn_kv, name=row.name, in_service=row.in_service)

    valid_buses = set(net.bus.index)
    print(f"✅ Added {len(valid_buses)} buses")

    # Step 2: 添加 line —— 自动跳过无效母线
    invalid_lines = []
    for i, row in grid_dict['line'].iterrows():
        f, t = int(row.from_bus), int(row.to_bus)
        if f not in valid_buses or t not in valid_buses:
            invalid_lines.append(i)
            continue
        try:
            pp.create_line_from_parameters(
                net, f, t,
                length_km=row.get("length_km", 1.0),
                r_ohm_per_km=row.get("r_ohm_per_km", 0.1),
                x_ohm_per_km=row.get("x_ohm_per_km", 0.1),
                c_nf_per_km=row.get("c_nf_per_km", 0.0),
                max_i_ka=row.get("max_i_ka", 1.0),
                in_service=row.get("in_service", True),
                name=row.get("name", f"line_{i}")
            )
        except Exception as e:
            invalid_lines.append(i)

    if invalid_lines:
        print(f"⚠️ Skipped {len(invalid_lines)} invalid lines with missing buses")

    print(f"✅ Added {len(net.line)} lines")

    # Step 3: 添加变压器（安全模式）
    for i, row in grid_dict['trafo'].iterrows():
        hv, lv = int(row.hv_bus), int(row.lv_bus)
        if hv not in valid_buses or lv not in valid_buses:
            continue
        try:
            pp.create_transformer_from_parameters(
                net, hv, lv,
                sn_mva=row.sn_mva,
                vn_hv_kv=row.vn_hv_kv,
                vn_lv_kv=row.vn_lv_kv,
                vk_percent=row.vk_percent,
                vkr_percent=row.vkr_percent,
                pfe_kw=row.get("pfe_kw", 0),
                i0_percent=row.get("i0_percent", 0),
                in_service=row.get("in_service", True),
                name=row.get("name", f"trafo_{i}")
            )
        except Exception:
            pass
    print(f"✅ Added {len(net.trafo)} transformers")

    # Step 4: 添加外部网
    if len(grid_dict['ext_grid']):
        eg = grid_dict['ext_grid'].iloc[0]
        bus_id = int(eg.bus)
        if bus_id not in valid_buses:
            bus_id = net.bus.index[0]
            print(f"⚠️ External grid bus not found, redirected to bus {bus_id}")
        pp.create_ext_grid(net, bus=bus_id, vm_pu=eg.get("vm_pu", 1.0), name="ext_grid")
    else:
        pp.create_ext_grid(net, bus=net.bus.index[0], vm_pu=1.0)
        print("⚠️ No ext_grid found — created temporary one")

    print(f"✅ Added {len(net.ext_grid)} external grid(s)\n")

    # Step 5: 清理重复/低阻抗线路
    dup_pairs = net.line.groupby(["from_bus", "to_bus"]).size()
    dup_pairs = dup_pairs[dup_pairs > 1]
    if len(dup_pairs):
        for (f, t), count in dup_pairs.items():
            idxs = net.line.index[(net.line.from_bus == f) & (net.line.to_bus == t)]
            net.line.drop(index=idxs[1:], inplace=True)
        print(f"🧹 Removed {len(dup_pairs)} duplicate line groups")

    low_imp = (net.line.r_ohm_per_km * net.line.length_km < 0.05) | \
              (net.line.x_ohm_per_km * net.line.length_km < 0.05)
    low_lines = net.line[low_imp]
    if len(low_lines):
        net.line.loc[low_lines.index, ["r_ohm_per_km", "x_ohm_per_km", "length_km"]] = 0.1
        print(f"⚙️ Fixed {len(low_lines)} low-impedance lines")

    # Step 6: 只保留与外部网连通的主网
    G = top.create_nxgraph(net, include_trafos=True)
    ccs = list(nx.connected_components(G))
    if len(ccs) > 1:
        ext_bus = int(net.ext_grid.bus.iloc[0])
        main_comp = [c for c in ccs if ext_bus in c][0]
        buses_to_keep = set(main_comp)
        net.bus = net.bus.loc[net.bus.index.isin(buses_to_keep)]
        net.line = net.line.loc[
            net.line.from_bus.isin(buses_to_keep) & net.line.to_bus.isin(buses_to_keep)
        ]
        print(f"🔗 Kept {len(buses_to_keep)} buses in main component")

    # Step 7: 尝试潮流
    print("\n⚡ Running power flow ...")
    try:
        pp.runpp(net, init='flat', tolerance_mva=1e-6, max_iteration=200)
        print("✅ Power flow converged!")
    except pp.LoadflowNotConverged as e:
        print("❌ Power flow failed to converge.")
    except Exception as e:
        print("❌ Other error:", e)

    return net


# -------------------------
# 使用：
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP_clean = D7_fix_v2(sb_code1)


🔧 Loading and repairing SimBench network: 1-HVMV-urban-2.203-0-no_sw

✅ Added 196 buses
⚠️ Skipped 113 invalid lines with missing buses
✅ Added 121 lines
✅ Added 0 transformers
⚠️ External grid bus not found, redirected to bus 0
✅ Added 1 external grid(s)

⚙️ Fixed 57 low-impedance lines
🔗 Kept 114 buses in main component

⚡ Running power flow ...
✅ Power flow converged!
